In [53]:
!pip install -q xlrd
!pip install lightfm

import pandas as pd
import numpy as np
import copy
import scipy.sparse as sp
import pickle

from lightfm import LightFM


In [54]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [55]:
input_raw_data_path = "/content/drive/MyDrive/dm_data/OnlineRetail.xlsx"
online_retail_sales = pd.read_excel(input_raw_data_path)
online_retail_sales.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [56]:
# Get dimensions
num_users, num_items = online_retail_sales['CustomerID'].nunique(), online_retail_sales['Description'].nunique()

In [57]:
print(online_retail_sales.nunique())

InvoiceNo      25900
StockCode       4070
Description     4223
Quantity         722
InvoiceDate    23260
UnitPrice       1630
CustomerID      4372
Country           38
dtype: int64


In [58]:
print(online_retail_sales['StockCode'].nunique())

4070


In [59]:
print(online_retail_sales['Description'].nunique())

4223


In [60]:
item_labels = np.array(online_retail_sales['Description'].unique())
item_label_to_index_map = {str(v): int(k) - 1 for k, v in enumerate(item_labels)}

cust_ids = np.array(online_retail_sales['CustomerID'].unique())
cust_id_to_index_map = {str(v): int(k) - 1 for k, v in enumerate(cust_ids)}

countries = np.array(online_retail_sales['Country'].unique())
country_to_index_map = {str(v): int(k) - 1 for k, v in enumerate(countries)}

In [61]:
def _build_interaction_matrix(rows, cols, online_retail_sales, item_label_to_index_map, cust_id_to_index_map, country_to_index_map):

    mat = sp.lil_matrix((rows, cols), dtype=np.int32)

    for index, row in online_retail_sales.iterrows():
        uid = cust_id_to_index_map[str(row['CustomerID'])]
        iid = item_label_to_index_map[str(row['Description'])]
        countryid = country_to_index_map[str(row['Country'])]
        mat[uid, iid] = countryid

    return mat.tocoo()

In [62]:
item_features = sp.identity(num_items, format="csr", dtype=np.float32)

In [63]:
data = _build_interaction_matrix(
        num_users, num_items, online_retail_sales, item_label_to_index_map, cust_id_to_index_map, country_to_index_map
    )

In [64]:
model = LightFM(learning_rate=0.05, loss='warp', no_components=64, item_alpha=0.001)
model.fit_partial(data, item_features=item_features, epochs=20 )

item_vectors = item_features * model.item_embeddings

In [65]:
DATA_PATH = "/content/drive/MyDrive/dm_data/"
with open(DATA_PATH + 'products.pickle', 'wb') as f:
    pickle.dump({"item_labels": item_labels,
        "customer_ids": cust_ids,
        "country_labels": countries, "vector": item_vectors}, f)


In [66]:
def load_data():
    with open(DATA_PATH + 'products.pickle', 'rb') as f:
        data = pickle.load(f)
    return data

data = load_data()
vectors = data["vector"]
data


{'country_labels': array(['United Kingdom', 'France', 'Australia', 'Netherlands', 'Germany',
        'Norway', 'EIRE', 'Switzerland', 'Spain', 'Poland', 'Portugal',
        'Italy', 'Belgium', 'Lithuania', 'Japan', 'Iceland',
        'Channel Islands', 'Denmark', 'Cyprus', 'Sweden', 'Austria',
        'Israel', 'Finland', 'Bahrain', 'Greece', 'Hong Kong', 'Singapore',
        'Lebanon', 'United Arab Emirates', 'Saudi Arabia',
        'Czech Republic', 'Canada', 'Unspecified', 'Brazil', 'USA',
        'European Community', 'Malta', 'RSA'], dtype=object),
 'customer_ids': array([17850., 13047., 12583., ..., 13298., 14569., 12713.]),
 'item_labels': array(['WHITE HANGING HEART T-LIGHT HOLDER', 'WHITE METAL LANTERN',
        'CREAM CUPID HEARTS COAT HANGER', ..., 'lost',
        'CREAM HANGING HEART T-LIGHT HOLDER',
        'PAPER CRAFT , LITTLE BIRDIE'], dtype=object),
 'vector': array([[ 0.15715317,  0.05075845,  0.16749544, ..., -0.04258499,
         -0.11535739,  0.11093401],
        [

In [67]:
!pip install faiss-gpu

In [68]:
import faiss
faiss.MatrixStats(vectors).comments.split("\n")


['analyzing 4223 vectors of size 64',
 'no NaN or Infs in data',
 'all vectors are distinct',
 'range of L2 norms=[0.318247, 2.16211] (0 null vectors)',
 'matrix contains no 0s',
 'no constant dimensions',
 'no dimension has a too large mean',
 'stddevs per dimension are in [0.0921002 0.139653]',
 '']

#Exhaustive Search Using Faiss

Faiss (Facebook AI Similarity Search) has several methods for similarity search. In this, instances are represented as vectors and these vectors can be compared using Euclidean distance method. Vectors are considered as similar if they have lowest euclidean distance between them. In this way, the similar vectors are extracted for the given vector.

IndexFlatL2 method uses brute-force way to search for similarities using euclidean disatance method.

Faiss has two implementations of this operation:
*   Direct implementation i.e., using brute-force approach to loop over all the vectors, to find the most similar elements.
*   Second implementation is similar to the first implementation but uses BLAS library to calculate distance efficiently (via matrix/matrix multiplication), which makes this faster compared to the first approach

Below is an example of first implementation.

In [69]:
data = load_data()
vectors = data["vector"]
item_labels = data["item_labels"]

In [70]:
vectors.shape

(4223, 64)

In [71]:
%%time
index = faiss.IndexFlatL2(vectors.shape[1])
index.add(vectors)

CPU times: user 1.35 ms, sys: 4 µs, total: 1.36 ms
Wall time: 823 µs


In [72]:
%%time
search_vector = vectors[70:71]
distances, indices = index.search(search_vector, 10)


CPU times: user 8.17 ms, sys: 0 ns, total: 8.17 ms
Wall time: 9.42 ms


In [73]:
out = [item_labels[i] for i in indices[0]]
print(f"The most similar products to {item_labels[70]} are:", *out, sep='\n- ')

The most similar products to PACK OF 60 DINOSAUR CAKE CASES are:
- PACK OF 60 DINOSAUR CAKE CASES
- PACK OF 60 PINK PAISLEY CAKE CASES
- LUNCH BOX WITH CUTLERY RETROSPOT 
- AIRLINE BAG VINTAGE TOKYO 78
- PACK OF 60 SPACEBOY CAKE CASES
- PACK OF 72 RETROSPOT CAKE CASES
- PACK OF 72 SKULL CAKE CASES
- CHEST OF DRAWERS GINGHAM HEART 
- ANTIQUE SILVER TEA GLASS ETCHED
- 3 PIECE SPACEBOY COOKIE CUTTER SET


#Vector Encoding using Product Quantization

Faiss loads the entire index (about vectors) into the RAM during the querying process, so exhaustive search works ONLY if the number of vectors OR it's dimensions, is not huge. In other words, it will be challenging to use exhaustive search for dataset with millions of vectors having huge number of dimensions. 

In this case, we can use Faiss with Product Quantizer's compression algorithm to compress the indexed vectors. In this approach, vector size will be encoded to the specified number of bytes. As the stored vector is compressed, the distance calculated between the vectors will be a approximate value instead of an exact value. So, the similarities created using this approach will be of approximate similarities.

In [74]:
data = load_data()
vectors = data["vector"]
item_labels = data["item_labels"]

In [75]:
%%time
quantizer = faiss.IndexFlatL2(vectors.shape[1])
index = faiss.IndexIVFPQ(quantizer, vectors.shape[1], 100, 8, 8)               
index.train(vectors)
index.add(vectors)

CPU times: user 1.59 s, sys: 35 ms, total: 1.63 s
Wall time: 871 ms


In [76]:
%%time
search_vector = vectors[70:71]
distances, indices = index.search(search_vector, 10)

CPU times: user 9.34 ms, sys: 4 µs, total: 9.34 ms
Wall time: 6.41 ms


In [77]:
out = [item_labels[i] for i in indices[0]]
print(f"The most similar products to {item_labels[70]} are:", *out, sep='\n- ')

The most similar products to PACK OF 60 DINOSAUR CAKE CASES are:
- PACK OF 60 DINOSAUR CAKE CASES
- PACK OF 60 PINK PAISLEY CAKE CASES
- AIRLINE BAG VINTAGE TOKYO 78
- LUNCH BOX WITH CUTLERY RETROSPOT 
- CHEST OF DRAWERS GINGHAM HEART 
- PACK OF 72 SKULL CAKE CASES
- ANTIQUE SILVER TEA GLASS ETCHED
- SET OF 12 MINI BUNNIES IN A BUCKET
- CHRISTMAS LIGHTS 10 SANTAS 
- PACK OF 72 RETROSPOT CAKE CASES


#Locality Sensitive Hashing (LSH)

FAISS provides another index known as LSH to solve ANN search. LSH groups similar data points (or vectors) into the same bucket, using a hash function. LSH uses a hash function that results in higher collisions instead of minimal collisions (which is a general case in hashmap though).

After grouping the similar data points, LHS will only compare the search vector with other vectors from the same group that search vector belongs to. In this way, LSH compares the search vector with few number of vectors instead of all the vectors.



In [78]:
data = load_data()
vectors = data["vector"]
item_labels = data["item_labels"]

From this [link](https://www.pinecone.io/learn/vector-indexes/), it seems like using "number of bits" for hashed vector as 16 times of vector's dimension will yeild 80% of recall performance for the model.

In [79]:
%%time
d = vectors.shape[1]
nbits = d*16 
index = faiss.IndexLSH(d, nbits)
index.add(vectors)

CPU times: user 201 ms, sys: 5 ms, total: 206 ms
Wall time: 159 ms


In [80]:
%%time
search_vector = vectors[70:71]
distances, indices = index.search(search_vector, 10)

CPU times: user 0 ns, sys: 816 µs, total: 816 µs
Wall time: 813 µs


In [81]:
out = [item_labels[i] for i in indices[0]]
print(f"The most similar products to {item_labels[70]} are:", *out, sep='\n- ')

The most similar products to PACK OF 60 DINOSAUR CAKE CASES are:
- PACK OF 60 DINOSAUR CAKE CASES
- PACK OF 60 PINK PAISLEY CAKE CASES
- LUNCH BOX WITH CUTLERY RETROSPOT 
- PACK OF 60 SPACEBOY CAKE CASES
- AIRLINE BAG VINTAGE TOKYO 78
- PACK OF 72 SKULL CAKE CASES
- CHEST OF DRAWERS GINGHAM HEART 
- PACK OF 72 RETROSPOT CAKE CASES
- 75 GREEN PETIT FOUR CASES
- CHOCOLATE THIS WAY METAL SIGN


#Hierarchical Navigable Small World Graphs

HNSW is an extension to navigable small world (NSW) graphs, where an NSW graph is a graph structure containing vertices connected by edges to their nearest neighbors.

Each node in the graph represents a vector point, and nodes are linked to other nodes that are close in space. 

With a graph data structure on the data set, approximate nearest neighbors can be identified using graph traversal methods. For a given query, we find its nearest neighbors by starting at a random point in the graph and computing its distance to the given query. From this entry point, graph will be explored by calculating the distance to the query for each newly visited node (until no closer point is found).

HNSW-based ANNS is the highest performing index.




In [82]:
data = load_data()
vectors = data["vector"]
item_labels = data["item_labels"]

In [83]:
%%time
d = vectors.shape[1]
M = 64
index = faiss.IndexHNSWFlat(d, M)
index.hnsw.efConstruction = 64
index.hnsw.efSearch = 32
index.add(vectors)

CPU times: user 1.08 s, sys: 3.89 ms, total: 1.08 s
Wall time: 603 ms


In [84]:
%%time
search_vector = vectors[70:71]
distances, indices = index.search(search_vector, 10)

CPU times: user 1.88 ms, sys: 0 ns, total: 1.88 ms
Wall time: 2.19 ms


In [85]:
out = [item_labels[i] for i in indices[0]]
print(f"The most similar products to {item_labels[70]} are:", *out, sep='\n- ')

The most similar products to PACK OF 60 DINOSAUR CAKE CASES are:
- PACK OF 60 DINOSAUR CAKE CASES
- PACK OF 60 PINK PAISLEY CAKE CASES
- LUNCH BOX WITH CUTLERY RETROSPOT 
- AIRLINE BAG VINTAGE TOKYO 78
- PACK OF 60 SPACEBOY CAKE CASES
- PACK OF 72 RETROSPOT CAKE CASES
- PACK OF 72 SKULL CAKE CASES
- CHEST OF DRAWERS GINGHAM HEART 
- ANTIQUE SILVER TEA GLASS ETCHED
- 3 PIECE SPACEBOY COOKIE CUTTER SET


#Trees and Forests

Annoy (Approximate Nearest Neighbors Oh Yeah) is being used as recommendation engine by Spotify.
Annoy constructs N binary trees, where each tree is independently constructed by recursively partitioning the data points. While searching for nearby neighbors, search process is carried out by travelling through tree nodes (starting from the root). 

In [86]:
!pip install annoy
import annoy

In [87]:
data = load_data()
vectors = data["vector"]
item_labels = data["item_labels"]

In [88]:
%%time
d = vectors.shape[1]
index = annoy.AnnoyIndex(d)
for i, vec in enumerate(vectors):
    index.add_item(i, vec.tolist())
index.build(5)

CPU times: user 49 ms, sys: 3.01 ms, total: 52.1 ms
Wall time: 36.6 ms


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: The default argument for metric will be removed in future version of Annoy. Please pass metric='angular' explicitly.
  


In [89]:
def query(index, vector, k):
    indices = index.get_nns_by_vector(vector.tolist(), k)
    return indices

In [90]:
%%time
search_vector = vectors[70]
indices = query(index, search_vector, 10)

CPU times: user 1.17 ms, sys: 0 ns, total: 1.17 ms
Wall time: 1.03 ms


In [91]:
out = [item_labels[i] for i in indices]
print(f"The most similar products to {item_labels[70]} are:", *out, sep='\n- ')

The most similar products to PACK OF 60 DINOSAUR CAKE CASES are:
- PACK OF 60 DINOSAUR CAKE CASES
- PACK OF 60 PINK PAISLEY CAKE CASES
- LUNCH BOX WITH CUTLERY RETROSPOT 
- AIRLINE BAG VINTAGE TOKYO 78
- PACK OF 60 SPACEBOY CAKE CASES
- PACK OF 72 SKULL CAKE CASES
- PACK OF 72 RETROSPOT CAKE CASES
- CHEST OF DRAWERS GINGHAM HEART 
- 75 GREEN PETIT FOUR CASES
- ANTIQUE SILVER TEA GLASS ETCHED
